In [399]:
# this file realize the function of transimiting weight through dma. number in ddr: 64*256 => bram 32*512   
# but still confused about why transfer 64bit a time since I set it as 32..
# note that ./hw/mvp_20201022/mvp.bit is the basic module
# note that ./hw/mvp_20201024/mvp.bit finish the compute part
import pynq
import time
import numpy as np
from hw.stalk_2 import StalkOverlay
ol = StalkOverlay('./hw/mvp_20201024/mvp.bit',download=True)

In [400]:
ol?

In [401]:
xlnk1 = pynq.Xlnk()

In [442]:
def datachange(data):
    if data >10000:
        data=-((int(bin(data^0b1111111111111111),2))+1)
        print(data,end=" ")
    else:
        print(data,end=" ")

In [403]:
def xlnk_create(d):
    d_xlnk = xlnk.cma_array(shape=(len(d),), dtype=np.uint64)
    for i in range(len(d)):
        d_xlnk[i] = d[i]
    return d_xlnk

def file_load(file_path, quiet=True, base=2):
    d = list()
    if file_path == '':
        print('    Empty file path ... skip ...')
        return d, -1
    with open(file_path) as f:
        lines = f.readlines()
        if not quiet:
            print('    File loading: %s ... including %d lines.' % (file_path, len(lines)))
        for i in range(len(lines)):
            line = lines[i].strip('\n')
            d.append(int(line, base))
        d_xlnk = xlnk_create(d)
    return d, d_xlnk

In [404]:
dc,dc_xlnk= file_load('./hw/buffer4.txt')

In [405]:
da,da_xlnk= file_load('./hw/buffer5.txt')

In [406]:
print(dc,dc_xlnx)

[363707893, 1011576454, 2352740480, 3038053909, 2715596661, 682604470, 4185149961, 3649211289, 1852681493, 3876606262, 1912006957, 428807661, 2680479718, 3640800994, 2166800704, 4263969935, 1751875143, 3726826219, 2696707999, 2581344537, 4268048816, 3885767429, 1290074724, 2933580673, 53851608, 3916420771, 923491711, 706872551, 1993643038, 625629901, 2830738520, 3321707698, 33537156, 3590837757, 352962573, 1501962242, 2098797148, 2800168409, 2150204400, 249440495, 620688223, 761938609, 3174162160, 4201578368, 3110698058, 1168051213, 2861967060, 3728480319, 491067424, 2120691925, 2059312814, 2442826245, 1038836640, 3940363393, 2005460814, 2687348475, 1826152167, 2739727671, 3478956293, 4274127402, 2904791043, 3689096405, 1649034419, 1195297792, 2113178989, 3362816492, 498310210, 2656336597, 184099511, 3434048981, 1936294684, 2054043895, 1047098400, 3304567548, 671108919, 1062404448, 3004155709, 3655137592, 2438037626, 2997187386, 540158175, 2473480927, 2209212083, 1763689414, 3502499166

In [407]:
ol.cdma.get_status()

2

In [408]:
print(da,da_xlnx)

[13319623460214931764, 9157905944963815397, 8491074562777950369, 51402794158300555, 9329191945953342623, 4807750418333765943, 2430345132795080733, 3715053254883169574, 5925804557127239895, 9874532160927136795, 12362257007784428624, 12680319137836609817, 11292605847926865069, 1745230143908168704, 14291957150187693721, 6168133833860106895, 1919006011890454333, 17023075770723393578, 4764866194358242965, 13801400714655341879, 11108782380560341141, 15593720496656266918, 17699215291107541002, 3580919775450985533, 17255582237602231873, 6002109023004165280, 14520828111387697869, 10759296811781938380, 5238188005485036816, 9136509241955220584, 7524227225496459030, 7707066870598829591, 17054671571795510131, 14731954476647946966, 65337860299266470, 13774406839244761071, 1457684825094369604, 10460391065317856559, 1745128079773505053, 14793508015910116286, 10654221351642978425, 6118159578555884486, 4331341082851983008, 17366178792655772373, 2254816284661524220, 11571479003870329001, 1225350520811825

In [409]:
ol.cdma.burst_p2h_a(ps_base=da_xlnk.physical_address,num=512)
time.sleep(1)

In [410]:
ol.stalk.read(0x200000)

2014445876

In [411]:
ol.cdma.burst_p2h_b(ps_base=da_xlnk.physical_address,b_offset=0x0,num=512)
time.sleep(1)

In [412]:
ol.stalk.read(0x300000)

2014445876

In [413]:
ol.cdma.burst_p2h_c(ps_base=da_xlnk.physical_address,c_offset=0x0,num=512)
time.sleep(1)

In [414]:
ol.stalk.read(0x400000)

2014445876

In [415]:
ol.cdma.burst_p2h_d(ps_base=da_xlnk.physical_address,d_offset=0x0,num=512)
time.sleep(1)

In [416]:
ol.stalk.read(0x100000)

2014445876

In [417]:
bin(ol.stalk.read(0x400000))

'0b1111000000100100000000100110100'

In [432]:
ol.stalk.write(0x500004,1)

In [433]:
bin(ol.stalk.read(0x500008))

'0b1111111110111000'

In [434]:
datachange(ol.stalk.read(0x500008))

-72 

In [435]:
bin(ol.stalk.read(0x500010))

'0b10111000110110001100111011110111'

In [429]:
bin(ol.stalk.read(0x500014))

'0b1111000000100100000000100110100'

In [430]:
bin(ol.stalk.read(0x500018))

'0b1111000000100100000000100110100'

In [431]:
bin(ol.stalk.read(0x50001C))

'0b1111000000100100000000100110100'

In [100]:
bin(ol.stalk.read(0x40004))+bin(ol.stalk.read(0x40000))

'0b101110001101100011001110111101110b1111000000100100000000100110100'

In [157]:
for i in range(1025):
    base = 0x40000
    base = base + 4*i
    print(i)
    print(bin(ol.stalk.read(base)))

0
0b1111000000100100000000100110100
1
0b10111000110110001100111011110111
2
0b10110000000000011001011111100101
3
0b1111111000101110110101011101111
4
0b11000011101111000010100010100001
5
0b1110101110101100101101101010011
6
0b10100110010001001001110110001011
7
0b101101101001111010010001
8
0b10001011111110111111010010011111
9
0b10000001011101111111001010100101
10
0b1101101100001010001000100110111
11
0b1000010101110001000111110001000
12
0b10010001110110011011000000011101
13
0b100001101110100101001000000101
14
0b10001010111110000101010100100110
15
0b110011100011101000010101000000
16
0b110011111101011100100011010111
17
0b1010010001111001010111111011001
18
0b11001111001110110111110000011011
19
0b10001001000010010110001010101110
20
0b100011000100110000010001010000
21
0b10101011100011111000111101000000
22
0b10011001010101101011000100011001
23
0b10101111111110011000101100010010
24
0b11010001011011111110010101101
25
0b10011100101101110110010100110000
26
0b10111010111100110100010000000000
27
0b1100

556
0b1100101101000001000000000001010
557
0b11110101101000000011111010001000
558
0b110000111011010111010000111101
559
0b110001101100011111101110010000
560
0b110100100011011001000001
561
0b11101111011110000010010010011111
562
0b11011010101001000010010100000
563
0b1010011010010111100011001011011
564
0b10001111110001010010001011001101
565
0b11001001100001000101100000111011
566
0b11110011010000010101110011001100
567
0b10010101010100001011001101110001
568
0b11100101101001011100010100010000
569
0b1001000101100011100100000111100
570
0b10001101110100000111110001101000
571
0b1111110110010110110011010111111
572
0b11101001000010100001011100010110
573
0b1101000011010110110110011001011
574
0b101010001110011000011000010111
575
0b1101010111101010000000001111100
576
0b1011101110100110000001101110011
577
0b11101100101011100101110101101111
578
0b10011110100111001101011010110
579
0b11001100011100100110101010001011
580
0b10001100001011001100110100110
581
0b111010000010000001110000
582
0b101010011100101001

806
0b10101001001000011011010100100011
807
0b10001100000100101100100110001
808
0b100010110101100011101110100111
809
0b11101001001100100110011110011111
810
0b11110110110010100001110000100
811
0b10100010011000001000110010111111
812
0b11111001111100000000101000100001
813
0b10011001111011100010110110110110
814
0b11011111010001010100110010100100
815
0b10011010101111010010100000101111
816
0b1110001000110111100100101011
817
0b11010000110111110000001001010110
818
0b10001010001100010111001000011010
819
0b11011111110111110100000000110000
820
0b1010000001110100111100110010100
821
0b1100111000111001100000110010011
822
0b1000010111001110001101010010000
823
0b1000110110100010110000110000000
824
0b10100001101001000101111110011000
825
0b10111101101111011000011110110110
826
0b100110100010111100011110000
827
0b1101101011100111001111010100111
828
0b10111011011011110000011100111
829
0b10101010000011001011111101011110
830
0b10001001010011111000110110101001
831
0b111110000110111100010111001111
832
0b1111110

In [79]:
print(da.index(2791012249))

255


In [446]:
for i in range(512):
    ol.stalk.write(0x500004,i)
    #print(i)
    datachange(ol.stalk.read(0x500008))

72
-72
12
-4
-28
92
12
-16
-96
52
72
-16
-52
40
36
36
8
-36
-40
-8
72
-60
28
-68
-88
52
-4
12
12
48
-84
0
-92
-68
-16
-56
-68
36
4
-40
-12
32
4
20
28
-28
4
-36
44
-32
36
52
-44
16
-24
68
16
-12
-4
-16
24
16
64
20
44
-76
0
8
36
12
16
-16
64
-12
-64
24
64
-20
0
-36
-76
-24
8
8
20
-4
4
-60
-16
52
-28
24
-24
-12
-88
52
20
-12
92
56
64
24
-44
-72
24
-8
40
36
80
44
-120
-24
-44
20
8
48
36
36
-44
-68
24
0
16
-40
76
-20
60
4
64
-40
-24
-12
92
0
-52
-32
-8
28
48
104
-92
52
-32
-56
-44
-20
-44
-36
32
-16
24
-8
68
-20
92
-8
64
-4
80
-12
112
36
96
-104
76
72
4
-32
-40
-32
44
-56
-8
-28
-12
-24
-24
16
-4
0
-12
28
-32
-16
-20
-52
88
-40
-24
-88
-8
28
48
40
16
-16
0
16
-48
-32
-28
-16
-24
-64
-24
-72
140
-24
-28
40
64
-20
-20
32
-16
-104
-8
12
100
32
48
-8
12
-24
32
24
60
32
28
52
-36
100
-28
48
8
20
0
12
44
-44
28
24
0
12
28
16
-40
-92
-32
8
-28
-40
-80
36
-28
0
64
20
64
-44
-68
28
-100
56
-56
-16
44
24
72
52
36
-64
-40
-72
-16
4
-32
36
44
12
-4
-40
-60
-28
-36
-76
-20
-4
-12
24
24
4
0
44
28
48
40
1